In [1]:
import numpy as np

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

In [5]:
import numpy as np
from datetime import datetime
class LogicGate:
    W = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    b = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    z = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    a = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    def __init__(self,gateName,xdata,tdata,i_node,h_node,hidden_layer,o_node,learning_rate,iteration_count):
        self.gateName = gateName
        if xdata.ndim == 1:
            self.xdata = xdata.reshape(-1,1)
            self.tdata = tdata.reshape(-1,1)
        elif xdata.ndim == 2:
            self.xdata = xdata.reshape(-1,2)
            self.tdata = tdata.reshape(-1,1)
        
        self.i_node = i_node
        self.h_node = h_node #은닉층의 노드의 개수
        self.o_node = o_node
        self.hidden_layer = hidden_layer #은닉층의 개수
        
        self.learning_rate = learning_rate
        self.iteration_count = iteration_count
        for i in range(self.hidden_layer+2):
            self.W[i] = locals()['W{}'.format(i)] = 0
            self.b[i] = locals()['b{}'.format(i)] = 0
            self.z[i] = locals()['z{}'.format(i)] = 0
            self.a[i] = locals()['a{}'.format(i)] = 0
        
        for i in range(self.hidden_layer+2):
            
            if i == 0: # 입력층 부분
                self.W[i] = np.random.rand(self.i_node,self.h_node)
                self.b[i] = np.random.rand(self.h_node)
            elif i == (hidden_layer+1): #출력층 부분
                self.W[i] = np.random.rand(self.h_node,self.o_node)
                self.b[i] = np.random.rand(self.o_node)
            else: #은닉층 부분
                self.W[i] = np.random.rand(self.h_node,self.h_node)
                self.b[i] = np.random.rand(self.h_node)
        
    def loss_func(self):
        delta = 1e-7
        
        for i in range(self.hidden_layer+2):
            
            if i == 0:
                self.z[i] = np.dot(self.xdata, self.W[i]) + self.b[i]
                self.a[i] = sigmoid(self.z[i])
            elif i == (self.hidden_layer + 1):
                self.z[i] = np.dot(self.a[i-1],self.W[i]) + self.b[i]
                y = self.a[i] = sigmoid(self.z[i])
            else:
                self.z[i] = np.dot(self.a[i-1],self.W[i]) + self.b[i]
                self.a[i] = sigmoid(self.z[i])
            #print("z[",i,"] = ",self.z[i],"a[",i,"] = ",self.a[i] )
        return -np.sum(self.tdata*np.log(y+delta) + (1-self.tdata)*np.log(1-y+delta))
            
    def predict(self,test_data):
        for i in range(self.hidden_layer+2):
            
            if i == 0:
                self.z[i] = np.dot(test_data, self.W[i]) + self.b[i]
                self.a[i] = sigmoid(self.z[i])
            elif i == (self.hidden_layer + 1):
                self.z[i] = np.dot(self.a[i-1],self.W[i]) + self.b[i]
                y = self.a[i] = sigmoid(self.z[i])
            else:
                self.z[i] = np.dot(self.a[i-1],self.W[i]) + self.b[i]
                self.a[i] = sigmoid(self.z[i])
        if y >= 0.5:
            result = 1
        else:
            result = 0
        return y,result
    
    def accuracy1(self,test_xdata,test_tdata):
        matched_list = []
        not_matched_list = []
        
        for index in range(len(test_xdata)):
            
            (real_val , logical_val) = self.predict(test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_result = len(matched_list) / len(test_xdata)
        
        print("Accuracy1 => ", accuracy_result)
        return matched_list, not_matched_list
    
    def accuracy2(self,test_data):
        self.test_xdata = test_data[:, :-1]
        self.test_tdata = test_data[:, -1]

        matched_list = []
        not_matched_list = []

        for index in range(len(self.test_xdata)):

            (real_val , logical_val) = self.predict(self.test_xdata[index])

            if logical_val == test_tdata[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)

        accuracy_result = len(matched_list) / len(test_xdata)

        print("Accuracy2 => ", accuracy_result)
        return matched_list, not_matched_list
    
    def train(self):
        f = lambda x : self.loss_func()
        start_time = datetime.now()
        for step in range(self.iteration_count):
            for i in range(self.hidden_layer+2):
                self.W[i] -= self.learning_rate * numerical_derivative(f,self.W[i])
                self.b[i] -= self.learning_rate * numerical_derivative(f,self.b[i])

            if (step % 400) == 0 :
                print("step = ", step, "error_val = ", self.loss_func())
        end_time = datetime.now()
        print("")
        print("elapsed_time : ", end_time - start_time)   

In [6]:
xdata = np.array([[0,0],[0,1],[1,0],[1,1]])
AND_tdata = np.array([0,0,0,1])
test_data = np.array([[0,0],[0,1],[1,0],[1,1]])

input_nodes = 2
hidden_nodes = 2
hidden_layers = 2
output_nodes = 1

lr = 1e-1
count = 10001
And_obj = LogicGate("AND_Gate",xdata,AND_tdata,input_nodes,hidden_nodes,hidden_layers,output_nodes,lr,count)
And_obj.train()

step =  0 error_val =  5.000866404048346
step =  400 error_val =  2.2496829411211503
step =  800 error_val =  2.249546576410686
step =  1200 error_val =  2.249447095202311
step =  1600 error_val =  2.249370484089414
step =  2000 error_val =  2.2493063089156533
step =  2400 error_val =  2.249245817304556
step =  2800 error_val =  2.249180296968932
step =  3200 error_val =  2.249099089880388
step =  3600 error_val =  2.2489863573776994
step =  4000 error_val =  2.2488146159869356
step =  4400 error_val =  2.2485297791649
step =  4800 error_val =  2.248011472444202
step =  5200 error_val =  2.2469495005298485
step =  5600 error_val =  2.244367902789298
step =  6000 error_val =  2.236078836896847
step =  6400 error_val =  2.1860743820080795
step =  6800 error_val =  0.5652892336995243
step =  7200 error_val =  0.07533919534206257
step =  7600 error_val =  0.03784315451232014
step =  8000 error_val =  0.025118765882206368
step =  8400 error_val =  0.01876913918360853
step =  8800 error_val 

In [7]:
print(And_obj.gateName, "\n")
test_data = np.array([ [0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 1] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 0, 0, 1])

(true_list, false_list) = And_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = And_obj.accuracy2(test_data)

print(true_list)
print(false_list)

AND_Gate 

Accuracy1 =>  1.0
Accuracy2 =>  1.0
[0, 1, 2, 3]
[]


In [8]:
xdata = np.array([[0,0],[0,1],[1,0],[1,1]])
XOR_tdata = np.array([0,1,1,0])
test_data = np.array([[0,0],[0,1],[1,0],[1,1]])

input_nodes = 2
hidden_1_nodes = 4
hidden_2_nodes = 2
output_nodes = 1

lr = 1e-1
count = 10001
XOR_obj = LogicGate("XOR_Gate",xdata,XOR_tdata,input_nodes,hidden_1_nodes,hidden_2_nodes,output_nodes,lr,count)
XOR_obj.train()

step =  0 error_val =  5.505495711689171
step =  400 error_val =  2.7724385632322925
step =  800 error_val =  2.7723029164260202
step =  1200 error_val =  2.7721550640711516
step =  1600 error_val =  2.7719879102622356
step =  2000 error_val =  2.771791917896455
step =  2400 error_val =  2.771553391010809
step =  2800 error_val =  2.771251382901846
step =  3200 error_val =  2.7708516753118104
step =  3600 error_val =  2.770293970055485
step =  4000 error_val =  2.76946149825053
step =  4400 error_val =  2.768097763477427
step =  4800 error_val =  2.765527506740564
step =  5200 error_val =  2.7593848331013913
step =  5600 error_val =  2.735698677994832
step =  6000 error_val =  2.4760731910532074
step =  6400 error_val =  2.0088629817873933
step =  6800 error_val =  1.9468191277852869
step =  7200 error_val =  1.9242668940896848
step =  7600 error_val =  1.8671233277151953
step =  8000 error_val =  0.15388638138770885
step =  8400 error_val =  0.04235881386976889
step =  8800 error_val 

In [9]:
print(XOR_obj.gateName, "\n")
test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 0])

(true_list, false_list) = XOR_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = XOR_obj.accuracy2(test_data)

print(true_list)
print(false_list)

XOR_Gate 

Accuracy1 =>  1.0
Accuracy2 =>  1.0
[0, 1, 2, 3]
[]
